In [197]:
# QUISPE SOLIZ MATIAS FERNANDO ----- INGENIERIA DE SISTEMAS ----- EXAMEN FINAL SIS420

# Pregunta 1: Implementar un algoritmo genético, que resuelva un problema especifico que su persona considere suficiente para demostrar la utilidad e importancia de esta tecnología.

# ---- Problema: Resolver el problema de la mochila 0/1. Encontrar la combinación óptima de 
# objetos de valor y peso dados, para maximizar el valor total, respetando la capacidad máxima de la mochila. ----

# Resolveremos el problema de la mochila haciendo uso de un algoritmo genetico, para ello continuamos con lo siguiente
# Para este fin haremos un algoritmo genetico de optimizacion, donde cada cromosoma en la poblacion se considera un OBJETO que puede o no caber en la mochila


# Para mi problema hago uso de los siguientes datos:
# OBJETO 1 con VALOR = 4 y PESO = 7
# OBJETO 2 con VALOR = 5 y PESO = 6
# OBJETO 3 con VALOR = 6 y PESO = 8
# OBJETO 4 con VALOR = 3 y PESO = 2
# La capacidad de la mochila es: 15 (hablando de peso)
# Donde la funcion objetivo es:
# Z = 4*X1+5*X2+6*X3+3*X4
# Una de las restricciones seria que la sumatoria de cada objeto multiplicado por su peso no exceda los 15 de la mochila:
# 7*X1+6*X2+8*X3+2*X4 <= 15
# La funcion FITNESS sera igual a la funcion objetivo
# Y el inviduo que viole la restriccion no sera considerado (pena de muerte) (PENALIZACION)

# Haremos uso de dos operadores geneticos: el cruce y la mutacion


# importamos el modulo random para generar numeros aleatorios
import random
# Especificamente haremos uso de la funcion random() para generar numero aleatorios entre 0 y 1
# Tambien la funcion randint para generar cromosomas aleatorios de 4 bits

# Comenzamos declarando la funcion fitness que mide el desempeño de acuerdo al objetivo que se desea optimizar
# Esta funcion nos sirve para identificar que tan apto es el cromosoma que nos llega para evaluar
def calcular_fitness(cromosoma):
    # El valor para los objetos del 1 al 4 es [4, 5, 6, 3] respectivamente
    # Por lo que podemos deducir nuestra funcion objetivo, deseando obtener los mejores valores de Z
    # Función objetivo: Z = 4*X1 + 5*X2 + 6*X3 + 3*X4
    valor = [4, 5, 6, 3]
    return sum(u * x for u, x in zip(valor, cromosoma))


# Declaramos aqui la funcion para generar un cromosoma aleatorio
def generar_cromosoma():
    # Generar un cromosoma aleatorio de 4 bits
    cromosoma = [random.randint(0, 1) for _ in range(4)]
    return cromosoma


# Aqui declaramos la funcion para el cruce 'realizar_cruce'
# Donde solamente ejecutamos el cruce si el valor aleatorio generado es menor a la probabilidad de cruce (0.98)
def realizar_cruce(padre1, padre2, ver_corte):
    # Si se lleva a cabo el cruce entonces se genera un nuevo valor aleatorio para determinar el corte del cruce
    valor_aleatorio = random.random()
    # Si el valor_aleatorio es menor a 0.33 entonces el corte se hara entre el objeto 1 y el objeto 2 
    if valor_aleatorio < 0.33:
        index_corte = 1
    # Si el valor_aleatorio se encuentra entre 0.33 y 0.66 entonces el corte se hara entre el objeto 2 y el objeto 3 
    elif valor_aleatorio < 0.66:
        index_corte = 2
    # Si el valor_aleatorio se encuentra entre 0.66 y 1 entonces el corte se hara entre el objeto 3 y el objeto 4 
    else:
        index_corte = 3

    # Gracias a esta variable podemos o no observar los cortes que se llevan a cabo en cada generacion
    if ver_corte:
        # Podemos ver los cromosomas sin el corte
        print("cromosomas sin corte:")
        print("Padre 1:", padre1)
        print("Padre 2:", padre2)

        # Realizamos el corte
        hijo1 = padre1[:index_corte] + padre2[index_corte:]
        hijo2 = padre2[:index_corte] + padre1[index_corte:]

        # Podemos ver los cromosomas con el corte
        print("cromosomas con corte:")
        print("Hijo 1:", hijo1)
        print("Hijo 2:", hijo2)
    else:
        # Realizamos el corte
        hijo1 = padre1[:index_corte] + padre2[index_corte:]
        hijo2 = padre2[:index_corte] + padre1[index_corte:]
    
    # Se retorna dos hijos despues del cruce en caso de llevarse a cabo
    return hijo1, hijo2


# Aqui declaramos la funcion para la mutacion, esta funcion se lleva a cabo para aquellos genes
# que obtuvieron un valor aleatorio menor a la probabilidad de mutacion (0.1)
# Entonces estos valores cambian su valor, es decir mutan, si el gen estaba con 1 pasa a 0 y viceversa
def realizar_mutacion(cromosoma, probabilidad_mutacion):
    for i in range(len(cromosoma)):
        valor_aleatorio = random.random()
        if valor_aleatorio < probabilidad_mutacion:
            # Cambiar 0 por 1 y viceversa
            cromosoma[i] = 1 - cromosoma[i]
    return cromosoma


# Esta es una funcion para seleccionar al padre de acuerdo a un criterio de la probabilidad acumulada
# cada gen tiene una probabilidad de ser seleccionado para llevarse en la mochila, esto se obtiene sumando todos los valores
# de fitness de cada cromosoma, luego dividiendo cada probabilidad individual de cada gen entre dicha sumatoria
# La probabilidad acumulada no es mas que ir acumulando las probabilidades obtenidas, hasta llegar a 1.00000
def seleccionar_padre(poblacion, probabilidad_acumulada):
    # Para seleccionar un padre primero se genera un numero aleatorio entre 0 y 1
    # De este numero por ejemplo un 0.4465 se toma el valor superior a este dentro de las probabilidades acumuladas de cada cromosoma
    # De esa manera se selecciona un padre, cuando obtenemos 2 continuamos con lo siguiente:
    valor_aleatorio = random.random()
    for i, prob in enumerate(probabilidad_acumulada):
        if valor_aleatorio <= prob:
            return poblacion[i]


# Esta funcion solamente sirve para validar que se cumpla la restriccion
# y que ningun cromosoma exceda el peso que no debe ser mayor a 15
def verificar_restriccion(cromosoma):
    # Restricción: suma de pesos de los genes no debe ser mayor a 15
    pesos = [7, 6, 8, 2]
    suma_pesos = sum(p * x for p, x in zip(pesos, cromosoma))
    # Devuelve True o False
    return suma_pesos <= 15


# En esta funcion hacemos los calculos necesarios para poder calcular las probabilidades de cada cromosoma
# Realizamos esto a partir de un array donde nos llegan los valores fitness del cromosoma en cuestion
def calcular_probabilidades_eleccion(fitness_poblacion):
    suma_fitness = sum(fitness_poblacion)
    probabilidades = [fitness / suma_fitness for fitness in fitness_poblacion]
    return probabilidades

# Esta funcion nos sirve para calcular la probabilidad acumulada, aquella que usamos para seleccionar a los padres
# Esto gracias a que nos llegan las probabilidades de los cromosomas, y en base a ellos calculamos la probabilidad acumulada
def calcular_probabilidad_acumulada(probabilidades):
    probabilidad_acumulada = [sum(probabilidades[:i+1]) for i in range(len(probabilidades))]
    return probabilidad_acumulada


# Esta funcion es la mas importante ya que es la que contiene la logica para generar una nueva poblacion a partir de una poblacion inicial
def generar_nueva_poblacion(poblacion_actual, probabilidad_cruce, probabilidad_mutacion):
    # Inicializamos un array vacio que contendra la nueva poblacion
    nueva_poblacion = []

    # Calculamos el fitness de cada cromosoma en la población
    fitness_poblacion = [calcular_fitness(cromosoma) for cromosoma in poblacion]

    # Calculamos las probabilidades de cada cromosoma de ser elegido
    probabilidades_eleccion = calcular_probabilidades_eleccion(fitness_poblacion)

    # Calculamos la probabilidad acumulada
    probabilidad_acumulada = calcular_probabilidad_acumulada(probabilidades_eleccion)

    # Ejecutaremos este ciclo hasta tener una poblacion de las mismas dimensiones que la poblacion inicial
    while len(nueva_poblacion) < len(poblacion_actual):
        # Llamamos a nuestra funcion para seleccionar a los padres
        padre1 = seleccionar_padre(poblacion_actual, probabilidad_acumulada)
        padre2 = seleccionar_padre(poblacion_actual, probabilidad_acumulada)

        # Generamos un valor aleatorio entre 0 y 1
        valor_aleatorio1 = random.random()
        # Comparamos el valor generado con la probabilidad de cruce
        if valor_aleatorio1 < probabilidad_cruce:
            # Si se cumple entonces llevamos a cabo el cruce y toda su logica detras
            hijo1, hijo2 = realizar_cruce(padre1, padre2, False)
        else:
            # Si no se cumple la condicion entonces simplemente asignamos los valores de los padres a los hijos
            hijo1, hijo2 = padre1[:], padre2[:]

        # Aplicamos nuestro operador genetico de la mutacion
        hijo1 = realizar_mutacion(hijo1, probabilidad_mutacion)
        hijo2 = realizar_mutacion(hijo2, probabilidad_mutacion)

        # Verificamos que cumplan con las restricciones
        # De ser asi entonces se añaden a la nueva población
        # (La poblacion si digamos es de 4, la nueva poblacion debe mantener la misma cantidad)
        if verificar_restriccion(hijo1):
            nueva_poblacion.append(hijo1)

        if len(nueva_poblacion) < len(poblacion_actual) and verificar_restriccion(hijo2):
            nueva_poblacion.append(hijo2)

    return nueva_poblacion


# Esta es una funcion para generar una poblacion inicial
# Verificando que inicialmente se cumplan todas las restricciones
def generar_poblacion_inicial(tamano_poblacion):
    poblacion = []
    while len(poblacion) < tamano_poblacion:
        cromosoma = generar_cromosoma()
        if verificar_restriccion(cromosoma):
            poblacion.append(cromosoma)
    return poblacion

# Aqui es donde configuramos los parámetros del algoritmo genético
num_generaciones = 10000
tamano_poblacion = 4
probabilidad_cruce = 0.98
probabilidad_mutacion = 0.1

# Creamos la población inicial
poblacion = generar_poblacion_inicial(tamano_poblacion)
print("Poblacion inicial: ")
print(poblacion)

# Ejecutar algoritmo genético
# Establecemos el mejor fitness en 0 para que vaya subiendo hasta obtener el mejor fitness
mejor_fitness = 0
# No establecemos de momento un mejor cromosoma, hasta que aparezca el primero y se vaya mejorando
mejor_cromosoma = None
# Hacemos un recuento de la cantidad de generaciones que son generados con nuestro algoritmo
cantidad_generaciones = 0

mejor_poblacion = None

# Hacemos uso de un try sxcept, ya que hubo casos donde se da una division por cero a causa de las probabilidades que dan 0
# eso significa que ya no se encontro un valor mejor, entonces si se da ese error paramos el algoritmo y mostramos el mejor cromosoma
try:
    # Generamos tantas generaciones como especifiquemos
    for generacion in range(num_generaciones):
        # Haciendo uso de nuestra poblacion inicial, generamos una nueva poblacion y asi sucesivamente hasta que termine el ciclo
        poblacion = generar_nueva_poblacion(poblacion, probabilidad_cruce, probabilidad_mutacion)

        # Aqui calculamos el fitness y encontramos el mejor cromosoma de la generación actual
        for cromosoma in poblacion:
            fitness = calcular_fitness(cromosoma)
            if fitness > mejor_fitness:
                mejor_fitness = fitness
                mejor_cromosoma = cromosoma
                mejor_poblacion = poblacion

        cantidad_generaciones += 1

        # Esta parte sirve para mostrar la información de la generación actual
        print(f"Generación {generacion+1}")
        print(f"Mejor fitness: {mejor_fitness}")
        print()

except ZeroDivisionError:
    # Mensaje en caso de un error durante el algoritmo
    print("Se produjo un error en la generación", cantidad_generaciones)
    print("Último mejor cromosoma encontrado:")
    print(mejor_cromosoma)
    print("Con un fitness de:", mejor_fitness)

# Mostramos finalmente los datos finales de nuestro algoritmo que resuelve el problema de la mochila
# Donde definimos la combinacion optima que no exceda los 15 de peso de la mochila, y donde se seleccionen los mejores objetos
print("Cantidad de generaciones:", cantidad_generaciones)
print("Mejor cromosoma encontrado:")
print(mejor_cromosoma)
print("Con un fitness de:", mejor_fitness)
print("Mejor Poblacion:")
print(mejor_poblacion)

Poblacion inicial: 
[[0, 0, 0, 1], [1, 0, 0, 1], [1, 0, 1, 0], [1, 1, 0, 1]]
Generación 1
Mejor fitness: 12

Generación 2
Mejor fitness: 12

Generación 3
Mejor fitness: 12

Generación 4
Mejor fitness: 12

Generación 5
Mejor fitness: 12

Generación 6
Mejor fitness: 12

Generación 7
Mejor fitness: 12

Generación 8
Mejor fitness: 12

Generación 9
Mejor fitness: 12

Generación 10
Mejor fitness: 12

Generación 11
Mejor fitness: 12

Generación 12
Mejor fitness: 12

Generación 13
Mejor fitness: 12

Generación 14
Mejor fitness: 12

Generación 15
Mejor fitness: 12

Generación 16
Mejor fitness: 12

Generación 17
Mejor fitness: 12

Generación 18
Mejor fitness: 12

Generación 19
Mejor fitness: 12

Generación 20
Mejor fitness: 12

Generación 21
Mejor fitness: 12

Generación 22
Mejor fitness: 12

Generación 23
Mejor fitness: 12

Generación 24
Mejor fitness: 12

Generación 25
Mejor fitness: 12

Generación 26
Mejor fitness: 12

Generación 27
Mejor fitness: 12

Generación 28
Mejor fitness: 12

Generaci